# Загрузка модулей

In [5]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.utils import shuffle
from sklearn.base import TransformerMixin
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import StandardScaler
from nltk.tokenize import TweetTokenizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import ADASYN, BorderlineSMOTE,RandomOverSampler,SMOTE
from imblearn.pipeline import make_pipeline, Pipeline
from time import time
import pylab as plt
import seaborn

# Загрузка данных

In [13]:
test_data=pd.read_csv('task-3-dataset.csv',sep=',',header = None, skiprows=1, names = ['text', 'y'])
train_data=pd.read_csv('reviews_irecommend.txt', sep='^([^;]+);', header = None, names = ['y', 'text'])
train_data.reset_index(drop=True, inplace=True)   
test_data = test_data.replace({'+':1, '-':0})

C:\Users\2\AppData\Local\Temp\ipykernel_10512\3236851464.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_data=pd.read_csv('reviews_irecommend.txt', sep='^([^;]+);', header = None, names = ['y', 'text'])


In [15]:
all_data = pd.concat((train_data, test_data))

# Модель

В качестве baseline используем CountVectorizer и логистическую регрессию с параметрами по умолчанию и оценим качество по кросс-валидации. В качестве метрики качества используем Accuracy

In [22]:
pipe=Pipeline([
    ("vectorizer",CountVectorizer()),
    ("classifier",LogisticRegression(max_iter=600))
])

cross_val=cross_val_score(pipe,all_data['text'], all_data['y'])
print("Средняя точность по кросс-валидации: %0.3f, среднеквадратичное отклонение %0.3f" % (np.mean(cross_val),np.std(cross_val)))

Средняя точность по кросс-валидации: 0.888, среднеквадратичное отклонение 0.012


# Сравним предсказание с размеченной выборкой

In [23]:
pipe.fit(all_data['text'], all_data['y'])

pipe.predict(test_data['text'])

array([0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1], dtype=int64)

In [24]:
test_data['y'].to_numpy()

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1], dtype=int64)